# Prepare CAMEOX inputs for a entanglement pair

**Version**: 0.8 (Mar 2022)

**Abstract**: Upstream CAMEOX pipeline 

**Changes**: Improved and extended after installing hh-suite; tackles problem with missing initial Met; includes initial tests and call to jackhmmer.

**Environment**: CZ jupyterhub on Mammoth. 

## Initialization
### Dependencies

In [2]:
import os
from pathlib import Path, PosixPath
import pwd
import sys
import time
from typing import Set, List, Union, Any, Dict, Tuple, Iterable, NewType, Optional, NamedTuple, Final

Check the python version

In [3]:
if not (sys.version_info.major == 3 and sys.version_info.minor >= 8):
  print('ERROR! This script requires Python 3.8 or higher.')
  print(f'You are using Python {sys.version_info.major}.{sys.version_info.minor}')
  sys.exit(1)

In [4]:
from Bio import SeqIO
from evcouplings.align.alignment import Alignment
from evcouplings.align.protocol import modify_alignment, cut_sequence
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import pandas as pd
import scipy

Check version of several libraries

In [5]:
print(f'Pandas version: {(pdver := pd.__version__)}')
if not ((pdvers := list(map(int, pdver.split('.'))))[0] >=1 and pdvers[1] >=1):
  print('\tERROR! This script requires pandas 1.1.0 or higher.')
  print(f'\tYou are using pandas {pdver}')
  sys.exit(2)
print(f'NumPy version: {np.__version__}')
print(f'SciPy version: {scipy.__version__}')
print(f'Matplotlib version: {mpl.__version__}')

Pandas version: 1.3.4
NumPy version: 1.21.4
SciPy version: 1.7.1
Matplotlib version: 3.2.0


### Type annotations

In [6]:
Id = NewType('Id', int)
Seq = NewType('Seq', str)

### General constants

In [8]:
USERNAME: Final[str] = pwd.getpwuid(os.getuid()).pw_name
GPUSERVER: Final[str] = 'pascal'  # Update this with your GPU machine at LC

###  Behaviour

In [7]:
debug: bool = False
verbose: bool = True
interactive: bool = True  # Disable in case of trouble storing the notebook with plotly rendered i-plots
          
def vprint(*arguments, **kargs) -> None:
    """Print only if verbose mode is enabled"""
    if verbose:
        print(*arguments, **kargs)

### Targets

In [8]:
#proteins: List[str] = ['infA_ecoli_GREMLIN',
#                       'prmC_pf5_uref100',
#                       'aroB_pf5_uref100',
#                       'infA_ecoli_uref100',
#                       'cysJ_pf5_uref100',
#                       'aroB_ecoli_uref100',
#                       'cysJ_ecoli_uref100',
#                       'aroB_ecoli_GREMLIN',
#                       'infA_pf5_uref100']
proteins: List[str] = ['cymR_pputida_uref100'] # Dante's
#proteins: List[str] = ['infA_pf5_ortho', 'aroB_pf5_ortho',
#                       'infA_pf5_ortho_bis', 'aroB_pf5_ortho_bis',
#                       'infA_pf5_ortho_tris', 'aroB_pf5_ortho_tris',]

## Initial tasks and jackhmmer
Tips:
* Place the CAMEOX repo in your home directory under 'cameos' directory or,
* Create a symlink in your home directory pointing to the group 'cameos' directory
* Set that location in the BASE constant below (if different from the options shown)

### Specific constants

In [9]:
# Databases
UNIREF100: Final[PosixPath] = Path('/usr/workspace/kpath/cameos/dbs/uniref100.fasta')
UNIREF90: Final[PosixPath] = Path('/usr/workspace/kpath/cameos/dbs/uniref90.fasta')
# CAMEOS/X
BASE: Final[PosixPath] = Path('/usr/workspace/kpath/cameos/')
#BASE: Final[PosixPath] = Path.home() / Path('cameos') 
MSAS_BASE: Final[PosixPath] = BASE / Path('CAMEOX/main/msas/')
# Jackhmmer
JCKHMM_SH: Final[PosixPath] = Path('jackhmmer.sh')
JCKHMM_BIN: Final[PosixPath] = Path('/usr/workspace/kpath/cameos/hmmer/bin')
JCKHMM_BTHRE: Final[float] = 0.5  # Jackhmmer bitscore threshold
JCKHMM_NITER: Final[int] = 5 # Jackhmmer num of iterations

### Test basic prerequisites

In [10]:
failure: bool = False
for protein in proteins:
    print(f'> Check input requirements for {protein}: ', end='')
    base_path = Path(protein)
    wt_path = Path(protein, 'wt.fa')
    if not base_path.is_dir():
        print('Missing dir!')
        failure = True
    elif not wt_path.is_file():
        print('Missing WT fasta file!')        
        failure = True
    else:
        print('OK!')
print('> Further checks... ', end='')
if not UNIREF100.is_file() or not UNIREF90.is_file():
    print('UniProt databases are missing!')
    failure = True    
if not JCKHMM_SH.is_file():
    print('Unable to find jackhmmer.sh!')
    failure = True
if not JCKHMM_BIN.is_dir():
    print('Jackhmmer bin dir is missing!')
    failure = True    
if not BASE.is_dir():
    print('CAMEOS base dir is missing!')
    failure = True    
if not MSAS_BASE.is_dir():
    print('CAMEOX msas subdir is missing!')
    failure = True    
if failure:
    raise FileNotFoundError('There are missing files/dirs needed for the pipeline')
else:
    print('OK!')

> Check input requirements for cymR_pputida_uref100: OK!
> Further checks... OK!


### Submit jackhmmer job to batch system in current machine

In [24]:
for protein in proteins:
    print(f'> Submitting jackhmmer job for {protein}...')    
    # USAGE: sbatch jackhmmer.sh <bin_dir> <prot_dir> <bitscore_threshold> <niter> <seqdb>
    sbatch_msg = !sbatch {JCKHMM_SH} {JCKHMM_BIN} {MSAS_BASE/Path(protein)} {JCKHMM_BTHRE} {JCKHMM_NITER} {UNIREF100}
    print(sbatch_msg)

> Submitting jackhmmer job for cymR_pputida_uref100...
['Submitted batch job 91561']


In [12]:
# Check the jobs are finished before proceeding further
squeue_msg = !squeue -u {USERNAME}
done:bool = False
print('\n'.join(squeue_msg))
while not done:
    if any('jackhmme' in line for line in squeue_msg): 
        print('.', end='', flush=True)
        time.sleep(20)
        squeue_msg = !squeue -u {USERNAME}
    else:
        done = True
        print('\n', '\n'.join(squeue_msg), '\nDONE!')   

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             90256    pbatch jackhmme  metagen  R       0:10      1 mammoth63
...............................................................................................................................................................................................................................
              JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
DONE!


## Postprocess MSA

### Specific constants

In [43]:
HHFILTER: Final[PosixPath] = BASE / Path('hhsuite/bin/hhfilter')

### Specific functions

In [44]:
def postMSA(prefix:PosixPath, fmt:str='stockholm', met_start=True):
    """Postprocess (filter) HMMER alignments"""

    target_fname: PosixPath = Path(prefix, 'wt.fa')
    msa_ext: str
    if fmt == 'stockholm':
        msa_ext = '.sto'
    elif fmt == 'fasta':
        msa_ext = '.fa'
    elif fmt == 'a3m':
        msa_ext = '.a3m'
    else:
        raise ValueError(f'Unknown format "{fmt}"!')
        
    msa_fname: PosixPath = Path(prefix, 'alignment').with_suffix(msa_ext)
    output_prefix: PosixPath = Path(prefix, prefix)
        
    # Load target/wt sequence from fasta file (typically wt.fa)
    tgseq = SeqIO.read(target_fname, format='fasta')
    
    # Get raw alignment from stockholm file
    with open(msa_fname) as fsto:
        ali_raw = Alignment.from_file(fsto, fmt)
    if met_start and tgseq.seq[0] == 'M': # Force M always in the 1st column if present in the WT    
        ali_raw = ali_raw.replace('-', 'M', columns=[0])
    print(f'INFO: Input MSA has {ali_raw.N} sequences and {ali_raw.L} columns')

    # center alignment around focus/search sequence
    focus_cols = np.array([c != "-" for c in ali_raw[0]])
    focus_ali = ali_raw.select(columns=focus_cols)
   
    if fmt == 'stockholm':
        assert len(tgseq.seq) == len(focus_ali[0]), (
            f'{len(focus_cols)} focus cols, expected {len(tgseq.seq)}')
    else:
        if len(tgseq.seq) != len(focus_ali[0]):
            print(f'WARNING! {len(focus_cols)} focus cols, expected {len(tgseq.seq)}')

    TARGET_SEQ_INDEX = 0
    REGION_START = 0
    kwargs = {
            'prefix': str(output_prefix),
            'seqid_filter': None,  
            # 'seqid_filter' corresponds to "threshold" in run_hhfilter (default: 95): Sequence identity
            #  threshold for maximum pairwise identity (between 0 and 100)
            'hhfilter': str(HHFILTER), 
            # 'hhfilter' corresponds to "binary" in run_hhfilter: Path to hhfilter binary
            'minimum_sequence_coverage': 50,  # Use integer in [0, 100] or real in [0.0, 1.0] (Chloe: 50; test: 98)
            'minimum_column_coverage': 50,  # Use integer in [0, 100] or real in [0.0, 1.0] (Chloe: 70; test: 0)
            # 'minimum_column_coverage' makes columns with too many gaps lowercase; 0 covers all columns.
            'compute_num_effective_seqs': False,
            'theta': 0.8,  # value of theta in computing sequence weights
    }
    mod_outcfg, ali = modify_alignment(
        focus_ali, TARGET_SEQ_INDEX, tgseq.id, REGION_START, **kwargs)
    print(f'INFO: Output MSA has {ali.N} sequences and {ali.L} columns')

In [10]:
def reformat_msa(protein: str, srcfmt='a3m', tgtfmt='fasta') -> None:
    """Convert MSA format from source to target"""
    
    src_path = Path(protein, protein).with_suffix('.' + srcfmt)
    tgt_path = Path(protein, protein).with_suffix('.' + tgtfmt)

    with open(src_path) as src, open(tgt_path, 'w') as tgt:
        alig = Alignment.from_file(src, srcfmt)
        vprint(f'INFO: Input MSA has {alig.N} sequences and {alig.L} columns')
        alig.write(tgt, format=tgtfmt, width=sys.maxsize)  # No width limit (for fasta)
        print(f'Convert {src_path} --> {tgt_path} OK!')

In [65]:
%%time
for protein in proteins:
    print(f'> Postprocessing alignment for {protein}...')
    for fmt in ['stockholm', 'fasta', 'a3m']:
        try:
            postMSA(protein, fmt=fmt)
        except FileNotFoundError:
            pass
        else:
            print(f'\tDone with alignment in {fmt} format for {protein}!')
            break

> Postprocessing alignment for infA_pf5_ortho...
INFO: Input MSA has 653 sequences and 72 columns
INFO: Output MSA has 653 sequences and 72 columns
	Done with alignment in a3m format for infA_pf5_ortho!
> Postprocessing alignment for aroB_pf5_ortho...
INFO: Input MSA has 620 sequences and 366 columns
INFO: Output MSA has 620 sequences and 366 columns
	Done with alignment in a3m format for aroB_pf5_ortho!
> Postprocessing alignment for infA_pf5_ortho_bis...
INFO: Input MSA has 3087 sequences and 72 columns
INFO: Output MSA has 3087 sequences and 72 columns
	Done with alignment in a3m format for infA_pf5_ortho_bis!
> Postprocessing alignment for aroB_pf5_ortho_bis...
INFO: Input MSA has 5492 sequences and 366 columns
INFO: Output MSA has 5492 sequences and 366 columns
	Done with alignment in a3m format for aroB_pf5_ortho_bis!
> Postprocessing alignment for infA_pf5_ortho_tris...
INFO: Input MSA has 5850 sequences and 98 columns
INFO: Output MSA has 5850 sequences and 72 columns
	Done wit

### Get MSAs in fasta format

In [66]:
for protein in proteins:
    protpath = Path(protein, protein)
    !mv -uv {protpath.with_suffix('.a2m')} {protpath.with_suffix('.a3m')}

‘infA_pf5_ortho/infA_pf5_ortho.a2m’ -> ‘infA_pf5_ortho/infA_pf5_ortho.a3m’
‘aroB_pf5_ortho/aroB_pf5_ortho.a2m’ -> ‘aroB_pf5_ortho/aroB_pf5_ortho.a3m’
‘infA_pf5_ortho_bis/infA_pf5_ortho_bis.a2m’ -> ‘infA_pf5_ortho_bis/infA_pf5_ortho_bis.a3m’
‘aroB_pf5_ortho_bis/aroB_pf5_ortho_bis.a2m’ -> ‘aroB_pf5_ortho_bis/aroB_pf5_ortho_bis.a3m’
‘infA_pf5_ortho_tris/infA_pf5_ortho_tris.a2m’ -> ‘infA_pf5_ortho_tris/infA_pf5_ortho_tris.a3m’
‘aroB_pf5_ortho_tris/aroB_pf5_ortho_tris.a2m’ -> ‘aroB_pf5_ortho_tris/aroB_pf5_ortho_tris.a3m’


In [ ]:
# Check WT sequence names and seqs
for protein in proteins:
    !head -2 {Path(protein, protein).with_suffix('.a3m')}

In [68]:
%%time
for protein in proteins:
    try:
        reformat_msa(protein, srcfmt='a3m', tgtfmt='fasta')
    except FileNotFoundError:
        if Path(protein, protein).with_suffix('.fasta').exists():
            print(f'WARNING! For {protein}, a3m file not found but fasta found!')
        else:
            print(f'ERROR! For {protein}, neither a3m or fasta file found!')
            raise

INFO: Input MSA has 653 sequences and 72 columns
Convert infA_pf5_ortho/infA_pf5_ortho.a3m --> infA_pf5_ortho/infA_pf5_ortho.fasta OK!
INFO: Input MSA has 620 sequences and 366 columns
Convert aroB_pf5_ortho/aroB_pf5_ortho.a3m --> aroB_pf5_ortho/aroB_pf5_ortho.fasta OK!
INFO: Input MSA has 3087 sequences and 72 columns
Convert infA_pf5_ortho_bis/infA_pf5_ortho_bis.a3m --> infA_pf5_ortho_bis/infA_pf5_ortho_bis.fasta OK!
INFO: Input MSA has 5492 sequences and 366 columns
Convert aroB_pf5_ortho_bis/aroB_pf5_ortho_bis.a3m --> aroB_pf5_ortho_bis/aroB_pf5_ortho_bis.fasta OK!
INFO: Input MSA has 5850 sequences and 72 columns
Convert infA_pf5_ortho_tris/infA_pf5_ortho_tris.a3m --> infA_pf5_ortho_tris/infA_pf5_ortho_tris.fasta OK!
INFO: Input MSA has 5554 sequences and 366 columns
Convert aroB_pf5_ortho_tris/aroB_pf5_ortho_tris.a3m --> aroB_pf5_ortho_tris/aroB_pf5_ortho_tris.fasta OK!
CPU times: user 5.05 s, sys: 19.2 ms, total: 5.07 s
Wall time: 5.1 s


## HMM branch

In [69]:
HMMBUILD = Path('/usr/workspace/kpath/cameos/hmmer/bin/hmmbuild')
HMMPRESS = Path('/usr/workspace/kpath/cameos/hmmer/bin/hmmpress')

### Specific functions

In [70]:
def hmmbuild(protein: str) -> None:
    """Wrapper for HMMER's hmmbuild"""
    
    protpath = Path(protein, protein)
    !{str(HMMBUILD)} --cpu 16 {protpath.with_suffix('.hmm')} {protpath.with_suffix('.fasta')} >> {protpath.with_suffix('.hmmbld.out')}
    if protpath.with_suffix('.hmm').exists():
        print(f'OK! hmmbuild was successful for {protein}')
    else:
        print(f'ERROR! hmmbuild output missing for {protein}')    

In [71]:
def hmmpress(protein: str) -> None:
    """Wrapper for HMMER's hmmpress"""
    
    protpath = Path(protein, protein)
    !{str(HMMPRESS)} -f {protpath.with_suffix('.hmm')} >> {protpath.with_suffix('.hmmpss.out')}
    if protpath.with_suffix('.hmm.h3m').exists() and protpath.with_suffix('.hmm.h3p').exists():
        print(f'OK! hmmpress was successful for {protein}')
    else:
        print(f'ERROR! hmmpress output missing for {protein}')    

### HMM build and compress

In [72]:
%%time
for protein in proteins:
    hmmbuild(protein)

OK! hmmbuild was successful for infA_pf5_ortho
OK! hmmbuild was successful for aroB_pf5_ortho
OK! hmmbuild was successful for infA_pf5_ortho_bis
OK! hmmbuild was successful for aroB_pf5_ortho_bis
OK! hmmbuild was successful for infA_pf5_ortho_tris
OK! hmmbuild was successful for aroB_pf5_ortho_tris
CPU times: user 33 ms, sys: 40.6 ms, total: 73.6 ms
Wall time: 3.18 s


In [73]:
for protein in proteins:
    hmmpress(protein)

OK! hmmpress was successful for infA_pf5_ortho
OK! hmmpress was successful for aroB_pf5_ortho
OK! hmmpress was successful for infA_pf5_ortho_bis
OK! hmmpress was successful for aroB_pf5_ortho_bis
OK! hmmpress was successful for infA_pf5_ortho_tris
OK! hmmpress was successful for aroB_pf5_ortho_tris


In [74]:
for protein in proteins:
    !cp {Path(protein, protein).with_suffix('.hmm.h??')} {Path('..', 'hmms')}
    !cp -v {Path(protein, protein).with_suffix('.hmm')} {Path('..', 'hmms')}

‘infA_pf5_ortho/infA_pf5_ortho.hmm’ -> ‘../hmms/infA_pf5_ortho.hmm’
‘aroB_pf5_ortho/aroB_pf5_ortho.hmm’ -> ‘../hmms/aroB_pf5_ortho.hmm’
‘infA_pf5_ortho_bis/infA_pf5_ortho_bis.hmm’ -> ‘../hmms/infA_pf5_ortho_bis.hmm’
‘aroB_pf5_ortho_bis/aroB_pf5_ortho_bis.hmm’ -> ‘../hmms/aroB_pf5_ortho_bis.hmm’
‘infA_pf5_ortho_tris/infA_pf5_ortho_tris.hmm’ -> ‘../hmms/infA_pf5_ortho_tris.hmm’
‘aroB_pf5_ortho_tris/aroB_pf5_ortho_tris.hmm’ -> ‘../hmms/aroB_pf5_ortho_tris.hmm’


## MRF branch

In [61]:
MSA2CCM = BASE / Path('CCMpred/scripts/convert_alignment.py')
CCMPRED = BASE /Path('CCMpred/bin/ccmpred')
JULIA = Path('/usr/gapps/julia/bin/julia-1.6.3') #was before: /usr/workspace/kpath/julia/julia
CCM2JLD = BASE / Path('CAMEOX/prepare/convert_ccm_to_jld.jl')
ENGYPSLS = BASE / Path('CAMEOX/prepare/energies_and_psls.jl')

### Specific functions

In [76]:
def msa2ccm(protein: str) -> None:
    """Wrapper for convert_alignment"""
    
    protpath = Path(protein, protein)
    !{str(MSA2CCM)} {protpath.with_suffix('.fasta')} fasta {protpath.with_suffix('.ccm')}
    if protpath.with_suffix('.ccm').exists():
        print(f'OK! msa2ccm was successful for {protein}')
    else:
        print(f'ERROR! msa2ccm output missing for {protein}')    

In [77]:
def ccmpred(protein: str, numiter:int=100, threads:int=32) -> None:
    """Wrapper for CCMpred"""
    
    protpath = Path(protein, protein)
    !{str(CCMPRED)} -r {protpath.with_suffix('.raw')} -n {numiter} -t {threads} {protpath.with_suffix('.ccm')} {protpath.with_suffix('.mat')} >> {protpath.with_suffix('.ccmprd.out')} 
    if protpath.with_suffix('.raw').exists():
        print(f'OK! CCMpred was successful for {protein}')
    else:
        print(f'ERROR! CCMpred raw file missing for {protein}')    

In [78]:
def ccm2jld(protein: str) -> None:
    """Wrapper for convert_ccm_to_jld"""
    
    protpath = Path(protein, protein)
    jld_path = Path('..', 'jlds', protein).with_suffix('.jld')
    !{str(JULIA)} {str(CCM2JLD)} {protpath.with_suffix('.raw')} {jld_path}
    if jld_path.exists():
        print(f'OK! ccm2jld was successful for {protein}')
    else:
        print(f'ERROR! ccm2jld output missing for {protein}')    

In [79]:
def engypsls(protein: str) -> None:
    """Wrapper for energy_and_psls"""
    
    print(f'\n>>> Sumarizing energy and psls for {protein}...')
    protpath = Path(protein, protein)
    jld_path = Path('..', 'jlds', protein).with_suffix('.jld')
    !{str(JULIA)} {str(ENGYPSLS)} {protein} {jld_path} {protpath.with_suffix('.fasta')}
    engy_path = Path('energy_' + protein).with_suffix('.txt')
    psls_path = Path('psls_' + protein).with_suffix('.txt')
    if engy_path.exists() and psls_path.exists():
        !mv -v {str(engy_path)} {str(Path('..', 'energies') / engy_path)}
        !mv -v {str(psls_path)} {str(Path('..', 'psls') / psls_path)}        
        print(f'OK! engypsls was successful for {protein}')
    else:
        print(f'ERROR! energy and/or psls output missing for {protein}')    

### Get the custom MSA

In [80]:
%%time
for protein in proteins:
    msa2ccm(protein)

OK! msa2ccm was successful for infA_pf5_ortho
OK! msa2ccm was successful for aroB_pf5_ortho
OK! msa2ccm was successful for infA_pf5_ortho_bis
OK! msa2ccm was successful for aroB_pf5_ortho_bis
OK! msa2ccm was successful for infA_pf5_ortho_tris
OK! msa2ccm was successful for aroB_pf5_ortho_tris
CPU times: user 20.9 ms, sys: 52.3 ms, total: 73.2 ms
Wall time: 1.75 s


### Training MRF —SLURM—

In [81]:
# Interactive run of CCMpred in the local machine (not recommended)
#for protein in proteins:
#    ccmpred(protein)

In [82]:
# Submit CCMpred with CUDA support to batch system in pascal (pvis queue limit 12h)
for protein in proteins:
    print(f'> Submitting CMMpred/CUDA job in pascal for {protein}...')    
    sbatch_msg = !sbatch CCMpred-CUDA_jobscript.sh {protein}
    print(sbatch_msg)

> Submitting CMMpred/CUDA job in pascal for infA_pf5_ortho...
['Submitted batch job 1421666 on cluster pascal']
> Submitting CMMpred/CUDA job in pascal for aroB_pf5_ortho...
['Submitted batch job 1421667 on cluster pascal']
> Submitting CMMpred/CUDA job in pascal for infA_pf5_ortho_bis...
['Submitted batch job 1421668 on cluster pascal']
> Submitting CMMpred/CUDA job in pascal for aroB_pf5_ortho_bis...
['Submitted batch job 1421669 on cluster pascal']
> Submitting CMMpred/CUDA job in pascal for infA_pf5_ortho_tris...
['Submitted batch job 1421670 on cluster pascal']
> Submitting CMMpred/CUDA job in pascal for aroB_pf5_ortho_tris...
['Submitted batch job 1421671 on cluster pascal']


In [83]:
# Check the jobs are finished before proceeding further
squeue_msg = !squeue -M pascal -u {USERNAME}
done:bool = False
print('\n'.join(squeue_msg))
while not done:
    if any('CCMpred' in line for line in squeue_msg):
        print('.', end='', flush=True)
        time.sleep(20)
        squeue_msg = !squeue -M pascal -u {USERNAME}
    else:
        done = True
        print('\n', '\n'.join(squeue_msg), '\nDONE!')

CLUSTER: pascal
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1421671      pvis CCMpred-  metagen PD       0:00      1 (Priority)
           1421670      pvis CCMpred-  metagen PD       0:00      1 (Priority)
           1421669      pvis CCMpred-  metagen PD       0:00      1 (Priority)
           1421668      pvis CCMpred-  metagen PD       0:00      1 (Priority)
           1421667      pvis CCMpred-  metagen PD       0:00      1 (Priority)
           1421666      pvis CCMpred-  metagen PD       0:00      1 (Priority)
.....
 CLUSTER: pascal
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
DONE!


### Convert the raw matrix to a JLD file

In [84]:
%%time
for protein in proteins:
    if Path(protein, protein).with_suffix('.raw').exists():
        ccm2jld(protein)
    else:
        print(f'ERROR! Raw (input) file not present for protein {protein}!!!')

OK! ccm2jld was successful for infA_pf5_ortho
OK! ccm2jld was successful for aroB_pf5_ortho
OK! ccm2jld was successful for infA_pf5_ortho_bis
OK! ccm2jld was successful for aroB_pf5_ortho_bis
OK! ccm2jld was successful for infA_pf5_ortho_tris
OK! ccm2jld was successful for aroB_pf5_ortho_tris
CPU times: user 695 ms, sys: 365 ms, total: 1.06 s
Wall time: 1min 51s


### Summarize energies and psls

#### Direct run

In [85]:
#%%time
#for protein in proteins:
#    engypsls(protein)

#### SLURM

In [86]:
for protein in proteins:
    print(f'> Submitting energy&psls job for {protein}...')
    sbatch_msg = !sbatch engypsls_jobscript.sh {protein}  # Uncomment this line for real submission
    print(sbatch_msg)

> Submitting energy&psls job for infA_pf5_ortho...
['Submitted batch job 86789 on cluster mammoth']
> Submitting energy&psls job for aroB_pf5_ortho...
['Submitted batch job 86790 on cluster mammoth']
> Submitting energy&psls job for infA_pf5_ortho_bis...
['Submitted batch job 86791 on cluster mammoth']
> Submitting energy&psls job for aroB_pf5_ortho_bis...
['Submitted batch job 86792 on cluster mammoth']
> Submitting energy&psls job for infA_pf5_ortho_tris...
['Submitted batch job 86793 on cluster mammoth']
> Submitting energy&psls job for aroB_pf5_ortho_tris...
['Submitted batch job 86794 on cluster mammoth']


In [87]:
# Check the jobs are finished before proceeding further
squeue_msg = !squeue -u {USERNAME}
done:bool = False
print('\n'.join(squeue_msg))
while not done:
    if any('engypsls' in line for line in squeue_msg): 
        print('.', end='', flush=True)
        time.sleep(20)
        squeue_msg = !squeue -u {USERNAME}
    else:
        done = True
        print('\n', '\n'.join(squeue_msg), '\nDONE!')   

CLUSTER: mammoth
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             86794    pbatch engypsls  metagen PD       0:00      1 (Resources)
             86793    pbatch engypsls  metagen PD       0:00      1 (Resources)
             86792    pbatch engypsls  metagen PD       0:00      1 (Resources)
             86791    pbatch engypsls  metagen PD       0:00      1 (Resources)
             86790    pbatch engypsls  metagen PD       0:00      1 (Priority)
             86789    pbatch engypsls  metagen PD       0:00      1 (Resources)
..................................
 CLUSTER: mammoth
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
DONE!


In [88]:
for protein in proteins:
    print(f'\n>>> Copying energy and psls files for {protein}...')
    engy_path = Path(protein, 'energy_' + protein).with_suffix('.txt')
    psls_path = Path(protein, 'psls_' + protein).with_suffix('.txt')
    if engy_path.exists() and psls_path.exists():
        !cp -v {str(engy_path)} {str(Path('..', 'energies'))}
        !cp -v {str(psls_path)} {str(Path('..', 'psls'))}        
    else:
        print(f'ERROR! energy and/or psls output missing for {protein}')  


>>> Copying energy and psls files for infA_pf5_ortho...
‘infA_pf5_ortho/energy_infA_pf5_ortho.txt’ -> ‘../energies/energy_infA_pf5_ortho.txt’
‘infA_pf5_ortho/psls_infA_pf5_ortho.txt’ -> ‘../psls/psls_infA_pf5_ortho.txt’

>>> Copying energy and psls files for aroB_pf5_ortho...
‘aroB_pf5_ortho/energy_aroB_pf5_ortho.txt’ -> ‘../energies/energy_aroB_pf5_ortho.txt’
‘aroB_pf5_ortho/psls_aroB_pf5_ortho.txt’ -> ‘../psls/psls_aroB_pf5_ortho.txt’

>>> Copying energy and psls files for infA_pf5_ortho_bis...
‘infA_pf5_ortho_bis/energy_infA_pf5_ortho_bis.txt’ -> ‘../energies/energy_infA_pf5_ortho_bis.txt’
‘infA_pf5_ortho_bis/psls_infA_pf5_ortho_bis.txt’ -> ‘../psls/psls_infA_pf5_ortho_bis.txt’

>>> Copying energy and psls files for aroB_pf5_ortho_bis...
‘aroB_pf5_ortho_bis/energy_aroB_pf5_ortho_bis.txt’ -> ‘../energies/energy_aroB_pf5_ortho_bis.txt’
‘aroB_pf5_ortho_bis/psls_aroB_pf5_ortho_bis.txt’ -> ‘../psls/psls_aroB_pf5_ortho_bis.txt’

>>> Copying energy and psls files for infA_pf5_ortho_tris..